In [ ]:
import cv2
import numpy as np
import socket

HOST     = "192.168.1.4"
PORT   = 20001

# Create a datagram socket
TCPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)

# Bind to address and ip
TCPServerSocket.bind((HOST, PORT))
TCPServerSocket.listen()

print("TCP server up and listening")
conn, addr = TCPServerSocket.accept()
# Listen for incoming datagrams/
while(True):
    msg = conn.recv(1024)
    if msg.decode() is not None:
        clientIP  = addr[0]
        print(f"Client {clientIP} connected")
        print("\t", msg.decode())

    if clientIP is None:
        pass
    
    # Sending a reply to client
    cap = cv2.VideoCapture(0)
    while(True):
        ret, frame = cap.read()
        if cap.isOpened() == 0:
            cap.open(0)

        # haar cascade for face detection
        model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        faces = model.detectMultiScale(frame)
        if (len(faces) != 0):
            for face in faces:
                x1 = face[0]
                y1 = face[1]
                x2 = x1 + face[2]
                y2 = y1 + face[3]
                img = cv2.rectangle(frame, (x1,y1), (x2,y2), [0,255,0], 3)
        else:
            img = frame

        img = cv2.putText(img, f"Faces Detected: {len(faces)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        r_img = cv2.resize(img, (300,200))
        data = r_img.tobytes()
        conn.send(data)

        